# Query ESGF input4MIPs index and extract source_id entries

## notes

## imports

In [2]:
%%time
import json
import numpy as np
import os
import pdb
import requests
from IPython.display import clear_output

CPU times: user 15 µs, sys: 2 µs, total: 17 µs
Wall time: 27.2 µs


## example source_id queries

## get source_id entries

In [3]:
# both input4MIPs and input4mips
project = "input4MIPs"

# Dataset search
inputsD = "https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true&" \
         "fq=type:Dataset&fq=replica:false&fq=project:" + project + "&facet.field=source_id"
js = requests.get(inputsD)
js_mipsD = json.loads(js.text)
print(inputsD)

# File search
inputsF = "https://esgf-node.llnl.gov/solr/files/select?q=*:*&rows=0&wt=json&facet=true&" \
          "fq=type:File&fq=replica:false&fq=project:" + project + "&facet.field=source_id"
js = requests.get(inputsF)
js_mipsF = json.loads(js.text)
print(inputsF)

https://esgf-node.llnl.gov/solr/datasets/select?q=*:*&rows=0&wt=json&facet=true&fq=type:Dataset&fq=replica:false&fq=project:input4MIPs&facet.field=source_id
https://esgf-node.llnl.gov/solr/files/select?q=*:*&rows=0&wt=json&facet=true&fq=type:File&fq=replica:false&fq=project:input4MIPs&facet.field=source_id


## extract source_id entries and composite

In [4]:
dicInpmD = js_mipsD["facet_counts"]["facet_fields"]["source_id"]
print("len(dicInpmD):", len(dicInpmD))
dicInpmF = js_mipsF["facet_counts"]["facet_fields"]["source_id"]
print("len(dicInpmF):", len(dicInpmF))
#print(dicInpmD)
#print(dicInpmF)

len(dicInpmD): 368
len(dicInpmF): 346


In [5]:
# datasets
srcIdLen = len(dicInpmD)
print("srcIdDLen:", srcIdLen)
els = np.arange(0, srcIdLen, 2)
srcIdDDict = {}
srcIds = dicInpmD  # reset to generic variable name
counts = 0
for cnt, srcId in enumerate(els.tolist()):
    #print(cnt, srcId)
    srcIdDDict[srcIds[srcId]] = srcIds[srcId+1]
    counts += int(srcIds[srcId+1])

print("len(srcIdDDict.keys()):", len(srcIdDDict.keys()))
print("dataset counts:", counts)
# sort dictionary
#sorted(srcIdDDict.items())
#print(srcIdDDict)
srcIdDDictList = list(sorted(srcIdDDict.keys()))
#print(srcIdDDictList)

# files
srcIdLen = len(dicInpmF)
print("srcIdFLen:", srcIdLen)
els = np.arange(0, srcIdLen, 2)
srcIdFDict = {}
srcIds = dicInpmF  # reset to generic variable name
counts = 0
for cnt, srcId in enumerate(els.tolist()):
    #print(cnt, srcId)
    srcIdFDict[srcIds[srcId]] = srcIds[srcId+1]
    counts += int(srcIds[srcId+1])

print("len(srcIdFDict.keys()):", len(srcIdFDict.keys()))
print("dataset counts:", counts)
# sort dictionary
srcIdFDictList = list(sorted(srcIdFDict.keys()))
#print(srcIdFDictList)

# determine missing
print("Search results: Dataset includes, excluded from File searches:")
set(srcIdDDictList).difference(srcIdFDictList)

srcIdDLen: 368
len(srcIdDDict.keys()): 184
dataset counts: 5876
srcIdFLen: 346
len(srcIdFDict.keys()): 173
dataset counts: 9373
Search results: Dataset includes, excluded from File searches:


{'CCMI-hist-nat-1-0',
 'CCMI-hist-sol-1-0',
 'CCMI-hist-volc-1-0',
 'IAMC-AIM-ssp370-1-0',
 'IAMC-GCAM4-ssp434-1-0',
 'IAMC-GCAM4-ssp460-1-0',
 'IAMC-IMAGE-ssp119-1-0',
 'IAMC-IMAGE-ssp126-1-0',
 'IAMC-MESSAGE-GLOBIOM-ssp245-1-0',
 'IAMC-REMIND-MAGPIE-ssp534-over-1-0',
 'IAMC-REMIND-MAGPIE-ssp585-1-0'}

## example source_id queries

## using source_id entries from Dataset search build a library

In [9]:
solrQry = "https://esgf-node.llnl.gov/esg-search/search/?limit=1000&format=application%2Fsolr%2Bjson&source_id=" \
          "PLACEHOLDER" + "&project=input4mips&project=input4MIPs&distrib=false&fields=*"  # all fields
mstrJson = {}  # create catch dictionary
oF = "tmp.json"
for count, srcId in enumerate(srcIdDDict.keys()):
    print(count, srcId)
    qryStr = solrQry.replace("PLACEHOLDER", srcId)
    #print("qryStr:", qryStr)
    js = requests.get(qryStr)
    js_srcId = json.loads(js.text)
    #print(js_srcId["response"]["docs"][0])
    # write to placeholder to test
    srcIdLen = len(js_srcId["response"]["docs"])
    clear_output(wait=False)
    # https://stackoverflow.com/questions/24816237/ipython-notebook-clear-cell-output-in-code
    print("len(js_srcId):", srcIdLen)
    for entry in np.arange(0, srcIdLen):
        a = js_srcId["response"]["docs"][entry]
        instId = a["instance_id"]
        mstrJson[instId] = a        
        if os.path.exists(oF):
            os.remove(oF)
        fH = open(oF, "w")
        json.dump(a, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
        #json.dump(mstrJson, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
        fH.close()
    #pdb.set_trace()
print("All done")
# cleanup
os.remove(oF)  # cleanup
# Write all out
oF = "comp.json"
if os.path.exists(oF):
    os.remove(oF)
fH = open(oF, "w")
json.dump(mstrJson, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
fH.close()

len(js_srcId): 247
All done


## create input4MIPs_source_id.json

In [ ]:
srcIdDict = {}
srcIdDict["source_id"] = {}
keyList = [
    "_timestamp", "contact", "dataset_category", "data_node", "datetime_start",
    "datetime_stop", "latest", "frequency", "institution_id", "mip_era", "realm",
    "replica", "source_id", "source_version", "target_mip", "version", "xlink"
       ]
# latest (bool)/deprecated (string)

# loop through entries
for c1, a in enumerate(mstrJson):
    tmp = mstrJson[a]
    srcId = tmp["source_id"][0]
    print(srcId)
    if srcId not in srcIdDict.keys():
        dic = [tmp.get(key) for key in keyList]
        srcIdDict["source_id"][srcId] = {}
        for c2, b in enumerate(keyList):
            c = dic[c2]
            if isinstance(c, list):
                c = c[0]
            srcIdDict["source_id"][srcId][keyList[c2]] = c
      
clear_output(wait=False)
print("len(srcIdDict[\"source_id\"].keys()):", len(srcIdDict["source_id"].keys()))
# Write all out
oF = "../input4MIPs_source_id.json"
if os.path.exists(oF):
    os.remove(oF)
fH = open(oF, "w")
json.dump(srcIdDict, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
fH.close()

## create other *.json entries

In [10]:
keyDict = {
    "DRS": {
        "directory_path_example": "input4MIPs/CMIP6Plus/CMIP/PCMDI/PCMDI-AMIP-1-1-9/ocean/mon/tos/gn/v20230512/",
        "directory_path_template": "<activity_id><mip_era>/<target_mip>/<institution_id>/<source_id>/<realm>/<frequency>/<variable_id>/<grid_label>/<version>",
        "filename_example": "tos_input4MIPs_SSTsAndSeaIce_CMIP_PCMDI-AMIP-1-1-9_gn_187001-202212.nc",
        "filename_template": "<variable_id><activity_id><dataset_category><target_mip><source_id><grid_label>[_<time_range>].nc"
    },
    "activity_id": {
        "input4MIPs": {
            "URL": "https://pcmdi.llnl.gov/mips/input4MIPs/",
            "long_name": "input forcing datasets for Model Intercomparison Projects"
        }
    },
    "dataset_category": [
        "GHGConcentrations",
        "SSTsAndSeaIce",
        "aerosolProperties",
        "atmosphericState",
        "emissions",
        "landState",
        "ozone",
        "radiation",
        "solar",
        "surfaceAir",
        "surfaceFluxes"
    ],
    "license" : {
        "license":"<your_data_identifier> data produced by <Your Institution; see MIP_institution_id.json> is licensed under a <Creative Commons; select and insert a license_id; see below> License (<insert the matching license_url; see below>). Consult https://pcmdi.llnl.gov/CMIP6Plus/TermsOfUse for terms of use governing CMIP6Plus and input4MIPs output, including citation requirements and proper acknowledgment. Further information about this data, can be found at <URL_maintained_data_provider>. The data producers and data providers make no warranty, either express or implied, including, but not limited to, warranties of merchantability and fitness for a particular purpose. All liabilities arising from the supply of the information (including any liability arising in negligence) are excluded to the fullest extent permitted by law.",
        "license_options":{
            "CC BY 4.0":{
                "license_id":"Creative Commons Attribution 4.0 International",
                "license_url":"https://creativecommons.org/licenses/by/4.0/"
            },
            "CC0 1.0":{
                "license_id":"Creative Commons CC0 1.0 Universal Public Domain Dedication",
                "license_url":"https://creativecommons.org/publicdomain/zero/1.0/"
            }
        }
    },
    "mip_era": [
        "CMIP5",
        "CMIP6",
        "CMIP6Plus"
    ],
    "product": [
        "derived",
        "observations",
        "reanalysis"
    ],
    "required_global_attributes": [
        "Conventions",
        "activity_id",
        "contact",
        "creation_date",
        "dataset_category",
        "frequency",
        "further_info_url",
        "grid_label",
        "institution",
        "institution_id",
        "license",
        "mip_era",
        "nominal_resolution",
        "realm",
        "region",
        "source",
        "source_id",
        "source_version",
        "table_id",
        "target_mip",
        "title",
        "tracking_id",
        "variable_id"
    ],
    "target_mip": {
        "CMIP6": {
            "AerChemMIP":"Aerosols and Chemistry Model Intercomparison Project",
            "C4MIP":"Coupled Climate Carbon Cycle Model Intercomparison Project",
            "CDRMIP":"Carbon Dioxide Removal Model Intercomparison Project",
            "CFMIP":"Cloud Feedback Model Intercomparison Project",
            "CMIP":"CMIP DECK: 1pctCO2, abrupt4xCO2, amip, esm-piControl, esm-historical, historical, and piControl experiments",
            "CORDEX":"Coordinated Regional Climate Downscaling Experiment",
            "DAMIP":"Detection and Attribution Model Intercomparison Project",
            "DCPP":"Decadal Climate Prediction Project",
            "DynVarMIP":"Dynamics and Variability Model Intercomparison Project",
            "FAFMIP":"Flux-Anomaly-Forced Model Intercomparison Project",
            "GMMIP":"Global Monsoons Model Intercomparison Project",
            "GeoMIP":"Geoengineering Model Intercomparison Project",
            "HighResMIP":"High-Resolution Model Intercomparison Project",
            "ISMIP6":"Ice Sheet Model Intercomparison Project for CMIP6",
            "LS3MIP":"Land Surface, Snow and Soil Moisture",
            "LUMIP":"Land-Use Model Intercomparison Project",
            "OMIP":"Ocean Model Intercomparison Project",
            "PAMIP":"Polar Amplification Model Intercomparison Project",
            "PMIP":"Palaeoclimate Modelling Intercomparison Project",
            "RFMIP":"Radiative Forcing Model Intercomparison Project",
            "SIMIP":"Sea Ice Model Intercomparison Project",
            "ScenarioMIP":"Scenario Model Intercomparison Project",
            "VIACSAB":"Vulnerability, Impacts, Adaptation and Climate Services Advisory Board",
            "VolMIP":"Volcanic Forcings Model Intercomparison Project"
        },
        "CMIP6Plus": {
            "CMIP": {
                "URL": "https://gmd.copernicus.org/articles/9/1937/2016/gmd-9-1937-2016.pdf",
                "long_name": "CMIP DECK: 1pctCO2, abrupt4xCO2, amip, esm-piControl, esm-historical, historical, and piControl experiments"
            },
            "LESFMIP": {
                "URL": "https://www.frontiersin.org/articles/10.3389/fclim.2022.955414/full",
                "long_name": "The Large Ensemble Single Forcing Model Intercomparison Project"
            }
        }
    },
    "tracking_id": [
        "hdl:21.14100/.*"
    ],
}
# Write all out
for key in keyDict.keys():
    print("processing:", key)
    oF = "".join(["../input4MIPs_", key, ".json"])
    print(oF)
    #pdb.set_trace()
    # add dict identifier
    tmp = {}
    tmp[key] = keyDict[key]
    # write to file
    if os.path.exists(oF):
        os.remove(oF)
    fH = open(oF, "w")
    json.dump(tmp, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
    fH.close()

processing: DRS
../input4MIPs_DRS.json
processing: activity_id
../input4MIPs_activity_id.json
processing: dataset_category
../input4MIPs_dataset_category.json
processing: license
../input4MIPs_license.json
processing: mip_era
../input4MIPs_mip_era.json
processing: product
../input4MIPs_product.json
processing: required_global_attributes
../input4MIPs_required_global_attributes.json
processing: target_mip
../input4MIPs_target_mip.json
processing: tracking_id
../input4MIPs_tracking_id.json


## create institution_id from source_id entries

In [14]:
# load data
srcIdPath = "../input4MIPs_source_id.json"
with open(srcIdPath, 'r') as fH:
    js_srcId = json.load(fH)

#js_srcId["source_id"].keys()
#js_srcId["source_id"]["ACCESS1-3-rcp85-1-0"]["institution_id"]

In [25]:
# iterate through and catch
instId = []
for count, key in enumerate(js_srcId["source_id"].keys()):
    instIdTmp = js_srcId["source_id"][key]["institution_id"]
    #print(count, key, instIdTmp)
    if instIdTmp not in instId:
        instId.append(instIdTmp)
instId.sort()
instId

['CCCma',
 'CNRM-Cerfacs',
 'IACETH',
 'IAMC',
 'ImperialCollege',
 'MOHC',
 'MPI-B',
 'MPI-M',
 'MRI',
 'NASA-GSFC',
 'NCAR',
 'NCAS',
 'PCMDI',
 'PNNL-JGCRI',
 'SOLARIS-HEPPA',
 'UCI',
 'UColorado',
 'UReading',
 'UoM',
 'UofMD',
 'VUA']

In [26]:
# write to input4MIPs_institution_id.json
tmp = {}
tmp["institution_id"] = instId
# Write all out
oF = "../input4MIPs_institution_id.json"
if os.path.exists(oF):
    os.remove(oF)
fH = open(oF, "w")
json.dump(tmp, fH, ensure_ascii=True, sort_keys=True, indent=4, separators=(",", ":"),)
fH.close()